In [1]:
from datasets import load_from_disk, Dataset, concatenate_datasets, ClassLabel, Features, Value, Sequence
from transformers import BertTokenizer, BertForSequenceClassification, BasicTokenizer
from torch.utils.data import  DataLoader
from tqdm.notebook import tqdm
import torch
import base
import os


[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/jovyan/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package punkt to /home/jovyan/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /home/jovyan/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger_eng to
[nltk_data]     /home/jovyan/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger_eng is already up-to-
[nltk_data]       date!


In [2]:
if torch.cuda.is_available():
    device = torch.device("cuda")
    print("GPU is available and will be used:", torch.cuda.get_device_name(0))
else:
    device = torch.device("cpu")
    print("GPU is not available, using CPU.")

GPU is available and will be used: NVIDIA H100 PCIe


In [ ]:
augmentation_params = {"n_iter": 5, "p_mask":0.1, "p_pos": 0.3, "p_ng":0.2}

In [ ]:
tokenizer = BasicTokenizer(do_lower_case=True)
DATASET = "sst2"

In [5]:
train_data = load_from_disk(f"~/data/{DATASET}/train")
sentences = list(map(lambda e: e["sentence"], train_data))

In [6]:
pos_tag_word_map_list = base.get_pos_tag_word_map(sentences, tokenizer=tokenizer)

In [ ]:
base.reset_seed()

In [8]:
augmented_datasets = base.get_augmented_dataset(augmentation_params, train_data, pos_tag_word_map_list, tokenizer=tokenizer)

In [9]:
aug_datasets_formated = []
ds_schema = Features({
    "idx": Value("int32"),
    "sentence": Value("string"),
    "label": ClassLabel(names=["negative", "positive"])
})

for iter in augmented_datasets:
    dataset = Dataset.from_dict(iter)
    dataset = dataset.cast(ds_schema)
    aug_datasets_formated.append(dataset)

Casting the dataset:   0%|          | 0/53879 [00:00<?, ? examples/s]

Casting the dataset:   0%|          | 0/53879 [00:00<?, ? examples/s]

Casting the dataset:   0%|          | 0/53879 [00:00<?, ? examples/s]

Casting the dataset:   0%|          | 0/53879 [00:00<?, ? examples/s]

Casting the dataset:   0%|          | 0/53879 [00:00<?, ? examples/s]

In [10]:
print(aug_datasets_formated[4][70])
print(train_data[70])

{'idx': 6575, 'sentence': "in reality it ' s [MASK] tough rock .", 'label': 1}
{'idx': 6575, 'sentence': "in reality it 's one tough rock . ", 'label': 1}


In [11]:
tokenizer = BertTokenizer.from_pretrained("gchhablani/bert-base-cased-finetuned-sst2")
model = BertForSequenceClassification.from_pretrained("gchhablani/bert-base-cased-finetuned-sst2", num_labels=2)
model.to(device)
model.eval()

torch.save(model.state_dict(), f"{os.path.expanduser('~')}/models/{DATASET}/teacher.pth")

tokenizer_config.json:   0%|          | 0.00/320 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/436k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/879 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/433M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/433M [00:00<?, ?B/s]

In [12]:
train_dataset = base.prepare_dataset(train_data, tokenizer)
train_dataloader = DataLoader(train_dataset, batch_size=128, shuffle=False)
train_logits = base.generate_logits(train_dataloader, model)
train_dataset = train_dataset.add_column("logits", train_logits)
train_dataset = train_dataset.remove_columns(["token_type_ids", "attention_mask"])
train_dataset.set_format(type="torch", columns=["logits", "labels"], device="cpu")

Tokenizing the provided dataset:   0%|          | 0/53879 [00:00<?, ? examples/s]

Generating logits for given dataset:   0%|          | 0/421 [00:00<?, ?it/s]

In [13]:
print(base.check_acc(train_dataset, "Accuracy for base dataset: "))

Calculating accuracy based on the saved logits:   0%|          | 0/53879 [00:00<?, ?it/s]

Accuracy for base dataset:  0.9888268156424581


In [14]:
aug_clean_datasets = []
for dataset in tqdm(aug_datasets_formated, total=(len(aug_datasets_formated)), desc="Processing augmented datasets: "):
    aug_train_dataset = base.prepare_dataset(dataset, tokenizer)
    aug_train_dataloader = DataLoader(aug_train_dataset, batch_size=128, shuffle=False)
    aug_train_logits = base.generate_logits(aug_train_dataloader, model)
    aug_train_dataset = aug_train_dataset.add_column("logits", aug_train_logits)
    aug_train_dataset = aug_train_dataset.remove_columns(["token_type_ids", "attention_mask"])
    aug_train_dataset.set_format(type="torch", columns=["logits", "labels"], device="cpu")

    aug_train_dataset = base.remove_diff_pred_class(train_dataset, aug_train_dataset, pytorch_dataset=False)
    
    aug_train_dataset.reset_format()
    aug_clean_datasets.extend(aug_train_dataset)

Processing augmented datasets:   0%|          | 0/5 [00:00<?, ?it/s]

Tokenizing the provided dataset:   0%|          | 0/53879 [00:00<?, ? examples/s]

Generating logits for given dataset:   0%|          | 0/421 [00:00<?, ?it/s]

Removing entries from augmented dataset that are different from the base one - based on saved logits:   0%|   …

Tokenizing the provided dataset:   0%|          | 0/53879 [00:00<?, ? examples/s]

...

Generating logits for given dataset:   0%|          | 0/421 [00:00<?, ?it/s]

Removing entries from augmented dataset that are different from the base one - based on saved logits:   0%|   …

Tokenizing the provided dataset:   0%|          | 0/53879 [00:00<?, ? examples/s]

Generating logits for given dataset:   0%|          | 0/421 [00:00<?, ?it/s]

Removing entries from augmented dataset that are different from the base one - based on saved logits:   0%|   …

In [15]:
train_dataset.reset_format()

In [16]:
print("First iter check")
print(aug_clean_datasets[1997])
filtered_base = train_dataset.filter(lambda x: x["idx"] == aug_clean_datasets[1997]["idx"])
print(filtered_base[0])

print("Second iter check")
print(aug_clean_datasets[66699])
filtered_base = train_dataset.filter(lambda x: x["idx"] == aug_clean_datasets[66699]["idx"])
print(filtered_base[0])


First iter check
{'idx': 20217, 'sentence': 'will be literally worth your time .', 'labels': 1, 'input_ids': [101, 1209, 1129, 6290, 3869, 1240, 1159, 119, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'logits': [-4.896267414093018, 4.807925701141357]}


Filter:   0%|          | 0/53879 [00:00<?, ? examples/s]

{'idx': 20217, 'sentence': 'will be well worth your time . ', 'labels': 1, 'input_ids': [101, 1209, 1129, 1218, 3869, 1240, 1159, 119, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'logits': [-4.8515143394470215, 4.748100280761719]}
Second iter check
{'idx': 34426, 'sentence': "the intensity with which he ' s willing to express his convictions", 'labels': 1, 'input_ids': [101, 1103, 7657, 1114, 1134, 1119, 112, 188, 4988, 1106, 6848, 1117, 22978, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'logits': [-3.8095197677612305, 3.8810811042785645]}


Filter:   0%|          | 0/53879 [00:00<?, ? examples/s]

{'idx': 34426, 'sentence': "the intensity with which he 's willing to express his convictions ", 'labels': 1, 'input_ids': [101, 1103, 7657, 1114, 1134, 1119, 112, 188, 4988, 1106, 6848, 1117, 22978, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'logits': [-3.8095197677612305, 3.8810811042785645]}


In [17]:
print(train_dataset.features)

{'idx': Value(dtype='int32', id=None), 'sentence': Value(dtype='string', id=None), 'labels': ClassLabel(names=['negative', 'positive'], id=None), 'input_ids': Sequence(feature=Value(dtype='int32', id=None), length=-1, id=None), 'logits': Sequence(feature=Value(dtype='float32', id=None), length=-1, id=None)}


In [18]:
ds_schema = Features({
    "idx": Value("int32"),
    "sentence": Value("string"),
    "labels": ClassLabel(names=["negative", "positive"]),
    "input_ids": Sequence(feature=Value(dtype="int32")),
    "logits": Sequence(feature=Value(dtype="float32")),
})

aug_dataset = Dataset.from_list(aug_clean_datasets)
aug_dataset = aug_dataset.cast(ds_schema)


Casting the dataset:   0%|          | 0/239755 [00:00<?, ? examples/s]

In [19]:
aug_dataset.set_format(type="torch", columns=["logits", "labels"], device="cpu")
train_dataset.set_format(type="torch", columns=["logits", "labels"], device="cpu")

In [20]:
print(base.check_acc(train_dataset, "Accuracy for base dataset: "))
print(base.check_acc(aug_dataset, "Accuracy for augmented dataset: "))

Calculating accuracy based on the saved logits:   0%|          | 0/53879 [00:00<?, ?it/s]

Accuracy for base dataset:  0.9888268156424581


Calculating accuracy based on the saved logits:   0%|          | 0/239755 [00:00<?, ?it/s]

Accuracy for augmented dataset:  0.9903151133448729


In [21]:
train_all_data = concatenate_datasets([train_dataset, aug_dataset])
train_all_data.set_format(type="torch", columns=["logits", "labels"], device="cpu")

In [22]:
print(base.check_acc(train_all_data, "Accuracy for combined dataset: "))

Calculating accuracy based on the saved logits:   0%|          | 0/293634 [00:00<?, ?it/s]

Accuracy for combined dataset:  0.9900420251060844


In [23]:
print(train_all_data.column_names)

['idx', 'sentence', 'labels', 'input_ids', 'logits']


In [24]:
train_all_data.reset_format()
train_all_data = train_all_data.remove_columns(["idx", "input_ids"])

In [25]:
train_all_data.save_to_disk(f"~/data/{DATASET}/train-logits-augmented")

Saving the dataset (0/1 shards):   0%|          | 0/293634 [00:00<?, ? examples/s]

In [26]:
train_dataset.reset_format()
train_dataset = train_dataset.remove_columns(["idx", "input_ids"])
train_dataset.save_to_disk(f"~/data/{DATASET}/train-logits")

Saving the dataset (0/1 shards):   0%|          | 0/53879 [00:00<?, ? examples/s]

In [27]:
eval_data = load_from_disk(f"~/data/{DATASET}/eval")
eval_dataset = base.prepare_dataset(eval_data, tokenizer)
eval_dataloader = DataLoader(eval_dataset, batch_size=128, shuffle=False)

Tokenizing the provided dataset:   0%|          | 0/872 [00:00<?, ? examples/s]

In [28]:
test_data = load_from_disk(f"~/data/{DATASET}/test")
test_dataset = base.prepare_dataset(test_data, tokenizer)
test_dataloader = DataLoader(test_dataset, batch_size=128, shuffle=False)

Tokenizing the provided dataset:   0%|          | 0/13470 [00:00<?, ? examples/s]

In [29]:
test_blank_data = load_from_disk(f"~/data/{DATASET}/test-blank")
test_blank_dataset = base.prepare_dataset(test_blank_data, tokenizer)
test_blank_dataloader = DataLoader(test_blank_dataset, batch_size=128, shuffle=False)

Tokenizing the provided dataset:   0%|          | 0/1821 [00:00<?, ? examples/s]

In [30]:
eval_logits = base.generate_logits(eval_dataloader, model)
test_logits = base.generate_logits(test_dataloader, model)
test_blank_logits = base.generate_logits(test_blank_dataloader, model)

Generating logits for given dataset:   0%|          | 0/7 [00:00<?, ?it/s]

Generating logits for given dataset:   0%|          | 0/106 [00:00<?, ?it/s]

Generating logits for given dataset:   0%|          | 0/15 [00:00<?, ?it/s]

In [31]:
eval_dataset.reset_format()
eval_dataset = eval_dataset.add_column("logits", eval_logits)
eval_dataset = eval_dataset.remove_columns(["idx", "token_type_ids", "input_ids", "attention_mask"])

In [32]:
test_dataset.reset_format()
test_dataset = test_dataset.add_column("logits", test_logits)
test_dataset = test_dataset.remove_columns(["idx", "token_type_ids", "input_ids", "attention_mask"])

In [33]:
test_blank_dataset.reset_format()
test_blank_dataset = test_blank_dataset.add_column("logits", test_blank_logits)
test_blank_dataset = test_blank_dataset.remove_columns(["idx", "token_type_ids", "input_ids", "attention_mask"])

In [34]:
eval_dataset.save_to_disk(f"~/data/{DATASET}/eval-logits")
test_dataset.save_to_disk(f"~/data/{DATASET}/test-logits")
test_blank_dataset.save_to_disk(f"~/data/{DATASET}/test-blank-logits")

Saving the dataset (0/1 shards):   0%|          | 0/872 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/13470 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/1821 [00:00<?, ? examples/s]

In [35]:
eval_data = load_from_disk(f"~/data/{DATASET}/eval-logits")
test_data = load_from_disk(f"~/data/{DATASET}/test-logits")

eval_data.set_format(type="torch", columns=["logits", "labels"], device="cpu")
test_data.set_format(type="torch", columns=["logits", "labels"], device="cpu")

print(base.check_acc(eval_data, "Accuracy for base eval dataset: "))
print(base.check_acc(test_data, "Accuracy for base test dataset: "))

Calculating accuracy based on the saved logits:   0%|          | 0/872 [00:00<?, ?it/s]

Accuracy for base eval dataset:  0.9231651376146789


Calculating accuracy based on the saved logits:   0%|          | 0/13470 [00:00<?, ?it/s]

Accuracy for base test dataset:  0.9896807720861173
